In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import shutil
import distutils.dir_util
import matplotlib.image as mpimg



In [2]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [14]:
image_generator_validation = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [15]:
#################################################################################################################
##beocat - '/homes/adalal/transfer_data10Classes/training'
##local - 'C:/Users/adalal/OneDrive - Kansas State University/PhD/Research/Dataset/transfer_data10Classes/training'
#################################################################################################################

train_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory= '/homes/adalal/transfer_data10Classes/training',
                                                 shuffle=True,
                                                 target_size=(224, 224), 
                                                 subset="training",
                                                 class_mode='categorical')

test_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory= '/homes/adalal/transfer_data10Classes/training',
                                                 shuffle=False,
                                                 target_size=(224, 224), 
                                                 subset="validation",
                                                 class_mode='categorical')



Found 5500 images belonging to 10 classes.
Found 1370 images belonging to 10 classes.


In [7]:
###########################################################
##beocat - '/homes/adalal/transfer_data10Classes/validation'
###########################################################
validation_dataset = image_generator_validation.flow_from_directory(batch_size=32,
                                                 directory='/homes/adalal/transfer_data10Classes/validation',
                                                 shuffle=False,
                                                 target_size=(224, 224), 
                                                 class_mode='categorical')


Found 687 images belonging to 10 classes.


In [8]:
train_dataset.num_classes

10

In [13]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [15]:

# load the ResNet-50 network, ensuring the head FC layer sets are left # off

print("[INFO] preparing model...")
baseModel = ResNet50V2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

#summary of the resnet50 model
#baseModel.summary()



[INFO] preparing model...


In [16]:
len(baseModel.layers)

190

In [17]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)


headModel = Dense(train_dataset.num_classes, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False

#summary of the resnet50 model    
model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_______________________________________________________________________________________

In [21]:
# compile the model

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

#adam = Adam(learning_rate=1e-4, decay=1e-4/20)
adam = Adam(learning_rate=0.001, decay=0.001/30)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
# train the model
print("[INFO] training model...")

H = model.fit_generator(
    train_dataset,
    steps_per_epoch = train_dataset.samples // 32,
    validation_data = validation_dataset, 
    validation_steps = validation_dataset.samples  // 32,
    epochs = 30,
    callbacks=[early_stopping]
)


[INFO] training model...
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
171/171 [==============================] - 1392s 8s/step - loss: 0.8611 - accuracy: 0.7101 - val_loss: 0.4527 - val_accuracy: 0.8378
Epoch 2/30
171/171 [==============================] - 1369s 8s/step - loss: 0.5473 - accuracy: 0.8114 - val_loss: 0.4242 - val_accuracy: 0.8378
Epoch 3/30
171/171 [==============================] - 1342s 8s/step - loss: 0.4893 - accuracy: 0.8319 - val_loss: 0.3994 - val_accuracy: 0.8571
Epoch 4/30
171/171 [==============================] - 1338s 8s/step - loss: 0.4204 - accuracy: 0.8511 - val_loss: 0.3723 - val_accuracy: 0.8631
Epoch 5/30
171/171 [==============================] - 1360s 8s/step - loss: 0.4079 - accuracy: 0.8561 - val_loss: 0.3585 - val_accuracy: 0.8720
Epoch 6/30
171/171 [==============================] - 1353s 8s/step - loss: 0.3815 - accuracy: 0.8628 - val_loss: 0.3626 - val_accuracy: 0.8646
Epoch 7/30
171/171 [===============

In [25]:
steps_per_epoch=(train_dataset.samples/32)
steps_per_epoch

171.875

In [26]:
# save it as a h5 file
model.save('model_resnet50V2_10classes_retest2023June.h5')

In [3]:
from tensorflow.keras.models import load_model
model = load_model('model_resnet50V2_10classes_retest2023June.h5')
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
############################################################################################################

In [ ]:
############################################################################################################################

In [ ]:
#############################################################################################
###################GETTING DESIRED LAYER FROM MODEL#########################################
#############################################################################################

In [4]:
##################to get the output of n layers or whole model#######################
layer_outputs = [layer.output for layer in model.layers[-3:-2]]
#layer_outputs = [layer.output for layer in model.layers[-3:-2]]
layer_outputs

[<tf.Tensor 'dense/Relu:0' shape=(None, 64) dtype=float32>]

In [5]:
layer_names = [layer.name for layer in model.layers[-3:-2]]
layer_names

['dense']

In [6]:
from tensorflow.keras.models import Model
feature_map_model = Model(inputs=model.input, outputs=layer_outputs)


In [7]:
feature_map_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_______________________________________________________________________________________

In [7]:
#############################################################################################
###################TO GET PREDICTIONS ON WHOLE MODEL#########################################
#############################################################################################

In [21]:
#prediction for test datasets
predIdxs = model.predict_generator(test_dataset, steps=(test_dataset.samples // 32) + 1)

Instructions for updating:
Please use Model.predict, which supports generators.


In [22]:
predIdxs.shape


(1370, 10)

In [23]:
predIdxs

array([[9.9999952e-01, 1.7004260e-07, 8.8059761e-11, ..., 4.5647890e-09,
        3.2312196e-14, 1.0358889e-10],
       [5.1954830e-01, 2.6020059e-01, 4.4453578e-04, ..., 5.0110058e-03,
        2.1637308e-03, 4.1104606e-04],
       [5.9796691e-01, 2.1950385e-02, 2.3139481e-02, ..., 2.8737138e-03,
        2.4025616e-01, 4.0774696e-02],
       ...,
       [7.3218400e-09, 2.5540210e-08, 6.3782157e-03, ..., 8.5131120e-08,
        7.4046693e-07, 9.9362034e-01],
       [4.2233773e-12, 2.2837515e-11, 2.5869613e-05, ..., 6.3378677e-11,
        4.8984213e-11, 9.9997413e-01],
       [1.4825530e-07, 1.3508283e-06, 2.3075353e-04, ..., 2.0814868e-07,
        1.9910220e-05, 9.9974483e-01]], dtype=float32)

In [24]:
# Get classes by max element in np (as a list)
classes = list(np.argmax(predIdxs, axis=1))
# Get filenames (set shuffle=false in generator is important)
filenames = test_dataset.filenames

In [25]:
import pandas as pd
df=pd.DataFrame(predIdxs)
df['filenames'] = filenames
print(df)

                 0             1             2             3             4  \
0     9.999995e-01  1.700426e-07  8.805976e-11  1.980769e-11  1.883064e-11   
1     5.195483e-01  2.602006e-01  4.445358e-04  3.220396e-04  4.757362e-04   
2     5.979669e-01  2.195038e-02  2.313948e-02  6.036312e-03  4.440014e-03   
3     9.967038e-01  2.883409e-04  2.915289e-06  1.048496e-05  6.391008e-06   
4     9.262685e-01  2.325485e-02  3.327389e-02  6.513204e-04  8.218689e-04   
...            ...           ...           ...           ...           ...   
1365  2.388190e-07  9.983758e-07  5.258842e-03  3.843665e-07  1.383712e-06   
1366  1.317786e-07  3.846806e-07  4.250472e-02  1.205445e-07  4.809168e-08   
1367  7.321840e-09  2.554021e-08  6.378216e-03  4.384682e-09  3.706236e-09   
1368  4.223377e-12  2.283751e-11  2.586961e-05  7.918127e-13  4.206423e-12   
1369  1.482553e-07  1.350828e-06  2.307535e-04  1.801118e-07  1.926732e-07   

                 5             6             7             8   

In [26]:
df.to_csv (r'predictions_of_tenNeurons_dataframe_retest2023June.csv', index = None, header=True) 

In [27]:
label_map = (test_dataset.class_indices)

In [28]:
label_map

{'bathroom': 0,
 'bedroom': 1,
 'building_facade': 2,
 'conference_room': 3,
 'dining_room': 4,
 'highway': 5,
 'kitchen': 6,
 'living_room': 7,
 'skyscraper': 8,
 'street': 9}

In [29]:
import pandas as pd
b = [(x[0], y) for x in  zip(classes, filenames) for y in x[1:]]
df = pd.DataFrame(b, columns=['Predicted_classes','image_name'])
print (df)
df.to_csv (r'finalClass_with_image_name_retest2023June.csv', index = None, header=True) 

      Predicted_classes                       image_name
0                     0  bathroom/ADE_train_00000006.jpg
1                     0  bathroom/ADE_train_00000007.jpg
2                     0  bathroom/ADE_train_00000008.jpg
3                     0  bathroom/ADE_train_00000009.jpg
4                     0  bathroom/ADE_train_00000010.jpg
...                 ...                              ...
1365                  9    street/ADE_train_00017260.jpg
1366                  9    street/ADE_train_00017261.jpg
1367                  9    street/ADE_train_00017262.jpg
1368                  9    street/ADE_train_00017263.jpg
1369                  9    street/ADE_train_00017264.jpg

[1370 rows x 2 columns]


In [30]:
pred_labels = np.argmax(predIdxs, axis=1)

In [31]:
from sklearn.metrics import classification_report
print("[INFO] evaluating network...")
print(classification_report(test_dataset.classes, pred_labels,
	target_names=test_dataset.class_indices.keys()))

[INFO] evaluating network...
                 precision    recall  f1-score   support

       bathroom       0.90      0.78      0.84       134
        bedroom       0.89      0.88      0.88       277
building_facade       0.68      0.60      0.64        45
conference_room       0.77      0.91      0.83        33
    dining_room       0.75      0.84      0.79        82
        highway       0.96      0.88      0.92        59
        kitchen       0.84      0.87      0.86       130
    living_room       0.76      0.74      0.75       139
     skyscraper       0.90      0.88      0.89        64
         street       0.92      0.96      0.94       407

       accuracy                           0.87      1370
      macro avg       0.84      0.83      0.83      1370
   weighted avg       0.87      0.87      0.87      1370



In [27]:
# plot the loss
plt.plot(H.history['loss'], label='train loss')
plt.plot(H.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_resnet50V2_unfreeze')

# plot the accuracy
plt.plot(H.history['accuracy'], label='train acc')
plt.plot(H.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_resnet50V2_unfreeze')

In [ ]:
#############################################################################################
###################ACTIVATIONS OF LAST DENSE LAYER###########################################
#############################################################################################

In [32]:
#prediction for test datasets
predIdxs = feature_map_model.predict_generator(test_dataset, steps=(test_dataset.samples // 32) + 1)

In [33]:
import pandas as pd
df=pd.DataFrame(predIdxs)
df['filenames'] = filenames
print(df)
df.to_csv (r'preds_of_64Neurons_denseLayer_1370Images_retest2023June.csv', index = None, header=True) 

             0         1    2        3    4    5         6         7  \
0     0.000000  0.000000  0.0  0.00000  0.0  0.0  0.806137  2.096591   
1     0.000000  0.000000  0.0  0.00000  0.0  0.0  3.065494  4.517578   
2     1.939141  0.000000  0.0  0.00000  0.0  0.0  0.000000  2.720087   
3     0.000000  0.000000  0.0  0.00000  0.0  0.0  1.210703  2.179142   
4     3.408473  2.001404  0.0  0.00000  0.0  0.0  1.517815  1.052417   
...        ...       ...  ...      ...  ...  ...       ...       ...   
1365  0.000000  3.760646  0.0  1.71156  0.0  0.0  2.180759  0.000000   
1366  0.028702  2.151206  0.0  0.00000  0.0  0.0  3.227870  0.000000   
1367  0.000000  5.952412  0.0  0.00000  0.0  0.0  4.714510  0.000000   
1368  0.000000  6.222051  0.0  0.00000  0.0  0.0  5.171289  0.000000   
1369  0.000000  1.753540  0.0  0.00000  0.0  0.0  1.772867  0.000000   

              8    9  ...   55        56        57        58        59   60  \
0     10.072591  0.0  ...  0.0  8.018784  8.471792  0.00

In [ ]:
####################predictions on OUTPUT-LAYER for Google_Images################################

In [70]:
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd

# List of new classes to test
#new_classes = ['bed', 'table', 'cushion']
new_classes = ['edifice and skyscraper']

# Directory containing the test images for the new classes
#test_directory = '/homes/adalal/transfer_data30Classes/training'
test_directory = '/homes/adalal/new_research_dataset_jun2023/google_dataset_top1_eciiConcept/neuron_63_and'

# Create an ImageDataGenerator for rescaling
rescale_generator = image.ImageDataGenerator(rescale=1./255)

# Load the test images for the new classes
test_images = []
filenames = []
class_names = []
for class_name in new_classes:
    class_directory = os.path.join(test_directory, class_name)
    for image_name in os.listdir(class_directory):
        image_path = os.path.join(class_directory, image_name)
        try:
            img = image.load_img(image_path, target_size=(224, 224))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            img = rescale_generator.standardize(img)
            test_images.append(img)
            filenames.append(image_name)
            class_names.append(class_name)
        except:
            print("Corrupt image:", image_name)

# Concatenate the test images into a single array
test_images = np.concatenate(test_images)

# Perform predictions for the new test images
predIdxs = model.predict(test_images)


# Get classes by max element in predIdxs (as a list)
classes = list(np.argmax(predIdxs, axis=1))

# Get filenames
#filenames = test_dataset.filenames

# Create a dataframe with predIdxs, filenames, and classes
df=pd.DataFrame(predIdxs)
df['Class_names'] = class_names
df['filenames'] = filenames
df['Predicted_classes'] = classes


print(df)
df.to_csv (r'neuron_63_and_googleImages_1stconcept_preds_retest.csv', index = None, header=True) 

                0             1         2             3             4  \
0    8.968083e-07  4.678070e-04  0.006586  3.958458e-07  1.106697e-05   
1    3.219937e-09  1.502136e-07  0.000102  5.352107e-09  2.693431e-07   
2    2.755841e-05  2.311802e-04  0.043594  1.169961e-05  3.352474e-05   
3    2.780879e-07  2.654728e-07  0.018854  1.438947e-07  3.993612e-08   
4    9.541921e-02  3.433600e-02  0.316397  7.658886e-03  8.869875e-04   
..            ...           ...       ...           ...           ...   
218  9.454956e-06  5.573157e-05  0.049089  6.258162e-05  4.103353e-05   
219  3.948339e-05  1.941067e-04  0.001001  2.359026e-05  8.563223e-06   
220  5.685420e-10  6.154236e-09  0.000006  1.498189e-08  1.801926e-08   
221  7.363840e-07  2.789847e-06  0.001924  1.806252e-05  4.296058e-07   
222  7.796931e-07  1.720025e-05  0.052372  1.338221e-06  8.864968e-06   

            5             6             7         8         9  \
0    0.000009  2.120709e-04  1.858390e-06  0.989960  0.002

In [25]:
predIdxs

array([[2.03886293e-06, 3.79114931e-06, 1.01986386e-01, ...,
        3.23193308e-06, 1.19855140e-04, 8.97883058e-01],
       [3.33790195e-09, 1.21119923e-08, 2.03699383e-04, ...,
        6.37851585e-08, 8.63037570e-08, 9.99795854e-01],
       [9.25770749e-09, 5.12016882e-08, 1.01979694e-03, ...,
        4.35672654e-08, 3.40163024e-06, 9.98935759e-01],
       ...,
       [2.15408472e-05, 1.11227146e-05, 9.87024903e-01, ...,
        1.51881231e-05, 6.05218171e-04, 1.21009247e-02],
       [1.19637280e-05, 6.86286103e-06, 2.95873672e-01, ...,
        5.31881269e-05, 4.34791655e-05, 7.04000950e-01],
       [2.04522098e-06, 6.08520986e-06, 3.35750222e-01, ...,
        8.47259798e-06, 2.96346261e-04, 6.63752556e-01]], dtype=float32)

In [ ]:
######################################CODE 3 WITH DENSE LAYER#####################

In [ ]:
###########################CODE for getting DENSE-LAYER activations of Google-Images#####################################

In [32]:
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd

# List of new classes to test
new_classes = ['pot_rack','whisk']

# Directory containing the test images for the new classes
#test_directory = '/homes/adalal/transfer_data30Classes/training'
test_directory = '/homes/adalal/new_research_dataset_jun2023/google_dataset_2nd_eciiConcept/neuron_58'

# Create an ImageDataGenerator for rescaling
rescale_generator = image.ImageDataGenerator(rescale=1./255)

# Load the test images for the new classes
test_images = []
filenames = []
class_names = []
for class_name in new_classes:
    class_directory = os.path.join(test_directory, class_name)
    for image_name in os.listdir(class_directory):
        image_path = os.path.join(class_directory, image_name)
        img = image.load_img(image_path, target_size=(224, 224))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = rescale_generator.standardize(img)
        test_images.append(img)
        filenames.append(image_name)
        class_names.append(class_name)

# Concatenate the test images into a single array
test_images = np.concatenate(test_images)

# Perform predictions for the new test images
predIdxs = feature_map_model.predict(test_images)


# Get classes by max element in predIdxs (as a list)
classes = list(np.argmax(predIdxs, axis=1))

# Get filenames
#filenames = test_dataset.filenames

# Create a dataframe with predIdxs, filenames, and classes
df=pd.DataFrame(predIdxs)
df['Class_names'] = class_names
df['filenames'] = filenames
df['Predicted_classes'] = classes


print(df)
df.to_csv (r'neuron_63_googleImages_denseLayer_retest.csv', index = None, header=True) 

            0         1    2         3    4    5         6         7  \
0    3.913355  0.000000  0.0  2.778746  0.0  0.0  0.000000  5.101858   
1    1.784468  0.000000  0.0  2.788860  0.0  0.0  0.000000  0.000000   
2    3.137875  0.000000  0.0  0.072998  0.0  0.0  1.166687  1.111390   
3    4.850747  0.000000  0.0  0.000000  0.0  0.0  0.000000  1.122530   
4    4.368125  0.000000  0.0  2.257786  0.0  0.0  0.000000  0.000000   
..        ...       ...  ...       ...  ...  ...       ...       ...   
398  2.873701  0.000000  0.0  0.327249  0.0  0.0  3.016097  0.000000   
399  5.048665  0.530976  0.0  0.000000  0.0  0.0  1.782853  0.326305   
400  1.890776  0.000000  0.0  0.000000  0.0  0.0  0.687273  0.000000   
401  3.864802  0.659192  0.0  0.000000  0.0  0.0  0.918369  0.000000   
402  0.570255  1.134125  0.0  1.112373  0.0  0.0  3.436224  0.000000   

            8    9  ...        57   58        59   60   61        62  \
0    0.169669  0.0  ...  0.000000  0.0  0.000000  0.0  0.0  0.0

In [ ]:
###########################CODE for getting DENSE-LAYER activations of Google-Images#####################################
##################################### WITH EVAL AND VALIDATION DATA#####################

In [71]:
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# List of new classes to test
new_classes = ['edifice and skyscraper']

# Directory containing the test images for the new classes
#test_directory = '/homes/adalal/transfer_data30Classes/training'
test_directory = '/homes/adalal/new_research_dataset_jun2023/google_dataset_top1_eciiConcept/neuron_63_and'

# Create an ImageDataGenerator for rescaling
rescale_generator = image.ImageDataGenerator(rescale=1./255)

# Load the test images for the new classes
test_images = []
filenames = []
class_names = []
for class_name in new_classes:
    class_directory = os.path.join(test_directory, class_name)
    for image_name in os.listdir(class_directory):
        image_path = os.path.join(class_directory, image_name)
        img = image.load_img(image_path, target_size=(224, 224))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = rescale_generator.standardize(img)
        test_images.append(img)
        filenames.append(image_name)
        class_names.append(class_name)

# Concatenate the test images into a single array
test_images = np.concatenate(test_images)

# Split the data into training and validation sets
train_images, val_images, train_filenames, val_filenames, train_class_names, val_class_names = train_test_split(
    test_images, filenames, class_names, test_size=0.2, random_state=42)

# Perform predictions for the new training images
train_predIdxs = feature_map_model.predict(train_images)
train_classes = list(np.argmax(train_predIdxs, axis=1))

# Perform predictions for the new validation images
val_predIdxs = feature_map_model.predict(val_images)
val_classes = list(np.argmax(val_predIdxs, axis=1))

# Create dataframes for training and validation sets
train_df = pd.DataFrame(train_predIdxs)
train_df['Class_names'] = train_class_names
train_df['Filenames'] = train_filenames
train_df['Predicted_classes'] = train_classes

val_df = pd.DataFrame(val_predIdxs)
val_df['Class_names'] = val_class_names
val_df['Filenames'] = val_filenames
val_df['Predicted_classes'] = val_classes

# Print the dataframes
print("evaluation set:")
print(train_df)
train_df.to_csv (r'neuron_63_and_googleImages_denseLayer_1stconcept_eval_retest.csv', index = None, header=True) 
print("verification set:")
print(val_df)
val_df.to_csv (r'neuron_63_and_googleImages_denseLayer_1stconcept_verif_retest.csv', index = None, header=True) 


evaluation set:
            0    1    2         3    4    5         6         7         8  \
0    5.694077  0.0  0.0  0.200516  0.0  0.0  2.193598  0.000000  0.000000   
1    4.558432  0.0  0.0  0.200104  0.0  0.0  0.000000  2.393600  2.180624   
2    4.681424  0.0  0.0  1.207755  0.0  0.0  0.000000  0.000000  0.000000   
3    4.230455  0.0  0.0  1.402725  0.0  0.0  0.000000  0.000000  0.000000   
4    1.635380  0.0  0.0  2.333380  0.0  0.0  0.445411  0.190478  0.000000   
..        ...  ...  ...       ...  ...  ...       ...       ...       ...   
173  2.728693  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000000  0.000000   
174  3.071405  0.0  0.0  1.644084  0.0  0.0  0.255156  0.900032  0.000000   
175  2.584795  0.0  0.0  0.274886  0.0  0.0  1.253585  0.000000  0.000000   
176  0.106364  0.0  0.0  4.150141  0.0  0.0  0.554191  0.000000  0.000000   
177  0.604958  0.0  0.0  5.494804  0.0  0.0  0.000000  3.149931  0.000000   

       9  ...   57   58   59   60   61        62        63 